In [31]:
import tensorflow as tf
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we can use again a convenient built-in function to import a graph_def into the 
    # current default Graph
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def, 
            return_elements=None, 
            name="prefix", 
            op_dict=None, 
            producer_op_list=None
        )
    return graph

# We use our "load_graph" function
graph = load_graph("./Model/output_graph.pb")

# We can verify that we can access the list of operations in the graph
for op in graph.get_operations():
    print(op.name)     # <--- printing the operations snapshot below
    # prefix/Placeholder/inputs_placeholder
    # ...
    # prefix/Accuracy/predictions

prefix/codes_input
prefix/embedding/embeddings
prefix/embedding/embeddings/read
prefix/embedding/Cast
prefix/embedding/embedding_lookup/axis
prefix/embedding/embedding_lookup
prefix/embedding/embedding_lookup/Identity
prefix/lambda_1/Sum/reduction_indices
prefix/lambda_1/Sum
prefix/numeric_input
prefix/catInp/concat/axis
prefix/catInp/concat
prefix/dropout_1/Identity
prefix/alpha/forward_lstm_1/kernel
prefix/alpha/forward_lstm_1/kernel/read
prefix/alpha/forward_lstm_1/recurrent_kernel
prefix/alpha/forward_lstm_1/recurrent_kernel/read
prefix/alpha/forward_lstm_1/bias
prefix/alpha/forward_lstm_1/bias/read
prefix/alpha/backward_lstm_1/kernel
prefix/alpha/backward_lstm_1/kernel/read
prefix/alpha/backward_lstm_1/recurrent_kernel
prefix/alpha/backward_lstm_1/recurrent_kernel/read
prefix/alpha/backward_lstm_1/bias
prefix/alpha/backward_lstm_1/bias/read
prefix/alpha/zeros_like
prefix/alpha/Sum/reduction_indices
prefix/alpha/Sum
prefix/alpha/ExpandDims/dim
prefix/alpha/ExpandDims
prefix/alpha/T

In [32]:
y_pred = graph.get_tensor_by_name("prefix/time_distributed_out/Reshape_1:0")

In [34]:
x = graph.get_tensor_by_name("prefix/codes_input:0")
z = graph.get_tensor_by_name("prefix/numeric_input:0")

In [35]:
import numpy as np
import pandas as pd
import random
from random import randint
from random import uniform
from sklearn.model_selection import train_test_split
import pickle

def get_rand_visit_codes():
    #num_of_codes_in_visit = randint(1,5)
    #visit_c = random.sample(range(0,68), num_of_codes_in_visit)
    num_of_codes_in_visit = 2
    visit_c = []
    #print (visit_c)
    return visit_c
ite_rand = 0
def get_rand_visit_numerics():
    #[Diastolic,Systolic, pulse]
    global ite_rand
    if ite_rand == 0:
        visit_n = [uniform(60,90),uniform(90,160),uniform(60,100)]
    else:
        visit_n = [uniform(90,110),uniform(160,180),uniform(100,120)]
    #print (visit_n)
    return visit_n

def create_patient():
    num_of_visits = randint(1,25)
    global ite_rand
    if ite_rand == 0:
        ite_rand = 1
    else:
        ite_rand = 0
    anomaly_patient_count = 0
    patient_c = []
    for k in range(0,num_of_visits,1):
        patient_c.append(get_rand_visit_codes())
    #print (patient_c)

    patient_n = []
    for k in range(0,num_of_visits,1):
        visit_numerics = get_rand_visit_numerics()
        patient_n.append(visit_numerics)
        anomaly_patient_count = anomaly_patient_count + validate_numerics_for_anomaly(visit_numerics)
        
    #print ("New patient "+str(len(target))+", # of visits: "+str(num_of_visits))
    #print ("Number of anomalies: "+ str(anomaly_patient_count))
    target = 0
    if anomaly_patient_count/num_of_visits > 0.7:
        target = 1
        #print ("Target: "+ str(1))
    else:
        target = 0
        #print ("Target: "+ str(0))
    #print("Target value in list: "+str(target[len(target)-1]))
    return [patient_c,patient_n,None,anomaly_patient_count, target]

def validate_numerics_for_anomaly(numerics):
    #[Diastolic,Systolic, pulse]
    if (numerics[0] >= 100 or numerics[0] <= 55):
        return 1
    elif (numerics[1] >= 170 or numerics[1] <= 85):
        return 1
    elif (numerics[2] >= 110 or numerics[2] <= 55):
        return 1
    else:
        return 0

In [36]:
patient_1 = create_patient()
print("patient_codes: "+ str(patient_1[0]))
print("patient_nums: "+ str(patient_1[1]))
print("patient_anomalies: " +str(patient_1[3]))
print("patient_target: " +str(patient_1[4]))

patient_codes: [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
patient_nums: [[93.3656672854318, 167.1588411453623, 115.55693992327657], [96.75340838713969, 171.67627944161126, 111.7863701716031], [101.56896180825011, 179.51969332631697, 103.28769752562644], [108.7169845028149, 167.442972299388, 112.83882869694926], [98.1528990398829, 173.48516808115227, 105.2275798676086], [102.1712911677258, 163.87643516799608, 116.33066627045238], [93.38428943183825, 168.66742933536668, 104.07357367113723], [102.59028196908736, 166.07805684514267, 109.26440134380749], [95.27194001757314, 163.2304573285089, 111.57840981239035], [94.21184392071572, 174.44999468946827, 101.51426842953266], [104.71391116960488, 170.23184576136316, 101.59398789617049], [107.05011026517758, 165.93337403760518, 107.39340198944662], [104.28564649093727, 178.27103153403596, 109.57340489599275], [90.90348383147747, 160.08548765897407, 105.25447092916262], [92.93097157497736, 172.9903373755861, 100.3110040

In [37]:
feed_dict_testing = {
    x: [patient_1[0]],
    z: [patient_1[1]]
}
sess= tf.Session(graph=graph)
result=sess.run(y_pred, feed_dict=feed_dict_testing)

In [38]:
print(result)

[[[0.93599236]]]


In [39]:
patient_2 = create_patient()
print("patient_codes: "+ str(patient_2[0]))
print("patient_nums: "+ str(patient_2[1]))
print("patient_anomalies: " +str(patient_2[3]))
print("patient_target: " +str(patient_2[4]))

patient_codes: [[], [], [], [], [], [], []]
patient_nums: [[67.34660913827794, 139.2099151758411, 60.01720842698034], [79.93993719368464, 136.67399427420975, 88.42142258567425], [63.729226221289096, 143.29577727724768, 68.20651629890907], [70.92886843275876, 104.18244370310514, 78.17991250987086], [69.14936195442698, 93.71556753146936, 67.95362574238099], [79.95802037973202, 103.92261829601448, 98.7553103135514], [69.50024461655006, 152.4788085502068, 77.79449494643013]]
patient_anomalies: 0
patient_target: 0


In [40]:
feed_dict_testing = {
    x: [patient_2[0]],
    z: [patient_2[1]]
}
sess= tf.Session(graph=graph)
result=sess.run(y_pred, feed_dict=feed_dict_testing)

In [41]:
print(result)

[[[0.24841169]]]


In [52]:
builder = tf.saved_model.builder.SavedModelBuilder("builder/1")

In [53]:
#y_pred = graph.get_tensor_by_name("time_distributed_out/Reshape_1:0")
#z = graph.get_tensor_by_name("numeric_input:0")
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    tensor_numeric_input = tf.saved_model.utils.build_tensor_info(z)
    tensor_codes_input = tf.saved_model.utils.build_tensor_info(x)

    # output tensor info
    tensor_info_output = tf.saved_model.utils.build_tensor_info(y_pred)

    # Defines the DeepLab signatures, uses the TF Predict API
    # It receives an image and its dimensions and output the segmentation mask
    prediction_signature = (
        tf.saved_model.signature_def_utils.build_signature_def(
            inputs={
                'numeric_input': tensor_numeric_input,
                'codes_input': tensor_codes_input
            },
            outputs={'time_distributed_out': tensor_info_output},
            method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))


    builder.add_meta_graph_and_variables(
        sess, [tf.saved_model.tag_constants.SERVING],
        signature_def_map={
            'predict_disease':
                prediction_signature,
        })

    # export the model
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: builder/1/saved_model.pb
